In [ ]:
# run this IN TERMINAL for the workspace"
"""
export HTTP_PROXY=http://httpproxy-tcop.vip.ebay.com:80
export HTTPS_PROXY=http://httpproxy-tcop.vip.ebay.com:80
pip install pyspark==3.1.1
"""


In [1]:
import bxkrylov

min_execs = 30

bxkrylov.spark_notebook(globals(),
                        spark_dynamicAllocation_minExecutors=min_execs,
                        spark_executor_instances=min_execs,
                        spark_dynamicAllocation_initialExecutors=min_execs,
                        spark_speculation=True,
                        spark_decommission_enabled=True,
                        spark_executor_memory="55g",
                       )



ModuleNotFoundError: No module named 'bxkrylov'

In [2]:
import datetime
import os
import pandas as pd
import functools

def get_madcdl_paths(from_date, to_date):
    base_dir = "/apps/b_adv/plmerch/cdl"
    cdl_table = "detail_gmv_part"
    date_format = "%Y/%m/%d"
    if os.environ.get("BX_UNITTEST"):
        base_dir = os.path.join(os.environ["BX_TEST_DATA"], "madcdl")
        date_format = "dt=%Y-%m-%d"  # for local test we save data partitioned by date using spark format
        cdl_table = ""  # in local test mode we do not save this sub-directory
    return [
        os.path.join(
            base_dir, datetime.datetime.strftime(date, date_format), cdl_table
        )
        for date in pd.date_range(from_date, to_date)
    ]


def union_all(dataframe_list, cols_to_select):
    u = None
    cols_set = functools.reduce(lambda a, b: a & b, (set(df.columns) for df in dataframe_list))
    cols_list = list(cols_set)
    for df in dataframe_list:
        if cols_to_select:
            df = df.select(*cols_to_select)
        else:
            df = df.select(*cols_list)
        if u is None:
            u = df
        else:
            u = u.union(df)
    return u


def get_madcdl_date_union(spark, from_date, to_date, cols_to_select=None, num_partitions=None):
    reader = spark.read
    if num_partitions:
        reader = reader.option("numPartitions", num_partitions)
    return union_all(
        [
            reader.load(path)
            for path in get_madcdl_paths(from_date, to_date)
        ],
        cols_to_select,
    )


In [3]:
madcdl = get_madcdl_date_union(spark, datetime.datetime(2023, 5, 25), datetime.datetime(2023, 5, 25))
madcdl.createOrReplaceTempView("madcdl")

# MFE 1% EP investigating module_id in HP_EVENT_FACT

In [58]:
experiment_data = spark.sql("""
select distinct
    PLMT,
    get_json_object(mbe_value, '$._po_repl_plmt') as replacement_id
from madcdl 
WHERE ICFBOT = '00'
AND experience NOT LIKE 'Apps%'
AND length(EUID) = 0
AND get_json_object(mbe_value, '$._po_repl_plmt') <> plmt
AND array_contains(XT_TAG, '249932')
AND DELIVERED_SURFACE_PLMT_IMP = '1'
""")
experiment_data.cache()
experiment_data.show()

+------+--------------+
|  PLMT|replacement_id|
+------+--------------+
|101111|        101113|
|101211|        101259|
|101423|        101251|
|101260|        101211|
|101279|        101485|
|101110|        101112|
|101430|        101437|
|101211|        101485|
|101112|        101110|
|101485|        101425|
|101439|        101437|
|101211|        101325|
|100725|        100721|
|101325|        101279|
|101429|        101436|
|101437|        101436|
|101438|        101429|
|101279|        101324|
|101430|        101439|
|101436|        101430|
+------+--------------+
only showing top 20 rows



## Placement -> module_id, instamce_id mapping

* Step 1: I created a mapping between placement_id -> module_id, instamce_id (representative, the maximum presented).

* Step 2: I sampled from the experiment data rows with replacements. 

* Step 3: join the experiment data with the mapping to add the module_ids and instance_ids.

* Step 4: join the experiment data with HP_EVENT_FACT on the replaced module_id, instance_id and check the percent that we do see the swapping on HP_EVENT_FACT.


In [4]:
#create mapping
hp_sql = """
with hp_grouped as (
select placement_id, module_id, instance_id, max(count) as max_n
from (
    select placement.placement_id as placement_id, mid as module_id, iid as instance_id, count(*) as count
    FROM (
        select dt, session_skey, guid, pageci, 
            signedin_uid as user_id, exprnc_name, site_id, event_timestamp, mstat.*
        from ACCESS_VIEWS.HP_EVENT_FACT
        lateral view explode(module_stats5(module_info, module_lst, navsrc_lst, viewdtls_lst, module_plmt_imprns)) as mstat
        where dt = "20230525" AND type = "same_day"
    ) ms
    lateral view outer explode(placements) as placement
    where placement.placement_id is not null and iid <> -1
    group by placement_id, module_id, instance_id
    )
group by placement_id, module_id, instance_id
)
SELECT placement_id, max(representative_module_id) as module_id, max(representative_instance_id) as instance_id
FROM (
select placement_id, FIRST(module_id) OVER (PARTITION BY placement_id ORDER BY max_n DESC) AS representative_module_id, 
                    FIRST(instance_id) OVER (PARTITION BY placement_id ORDER BY max_n DESC) as representative_instance_id
from hp_grouped
)
GROUP BY placement_id
"""
hp_grouped = spark.sql(hp_sql)
hp_grouped.cache()
hp_grouped.show()

+------------+---------+-----------+
|placement_id|module_id|instance_id|
+------------+---------+-----------+
|      101213|    46344|          1|
|      101526|   146925|          1|
|      101325|     5063|          2|
|      101278|    47528|          1|
|      101287|    47999|          1|
|      101100|     3021|          1|
|      100803|     4236|          1|
|      101524|   146925|          1|
|      101326|     5063|          3|
|      101259|    47269|          2|
|      101286|    47999|          1|
|      101666|     5481|          1|
|      101324|     5063|          1|
|      101251|    47269|          1|
|      101211|    46343|          1|
|      101485|    47529|          1|
|      101279|    47528|          2|
|      101280|    47528|          3|
|      101260|    47269|          3|
|      101423|     3021|          1|
+------------+---------+-----------+
only showing top 20 rows



In [5]:
hp_grouped.createOrReplaceTempView("hp_grouped")
experiment_data_with_madcdl = spark.sql("""
select user_id, replacement_id, PLMT as original_PLMT, module_id as replacement_module_id, instance_id as replacement_instance_id
from hp_grouped a inner join (
    select user_id, PLMT,
        get_json_object(mbe_value, '$._po_repl_plmt') as replacement_id
    from madcdl 
    WHERE ICFBOT = '00'
    AND experience NOT LIKE 'Apps%'
    AND length(EUID) = 0
    AND get_json_object(mbe_value, '$._po_repl_plmt') <> plmt
    AND array_contains(XT_TAG, '249932')
    AND DELIVERED_SURFACE_PLMT_IMP = '1'
    ) b on b.replacement_id = a.placement_id
    
""")
experiment_data_with_madcdl.cache()
experiment_data_with_madcdl.show()

+----------+--------------+-------------+---------------------+-----------------------+
|   user_id|replacement_id|original_PLMT|replacement_module_id|replacement_instance_id|
+----------+--------------+-------------+---------------------+-----------------------+
| 634081993|        101278|       101425|                47528|                      1|
| 802242081|        101278|       101425|                47528|                      1|
|  39423441|        101278|       101425|                47528|                      1|
|1099311390|        101260|       101259|                47269|                      3|
|1986943045|        101260|       101259|                47269|                      3|
|1979500804|        101260|       101259|                47269|                      3|
| 351759445|        101260|       101259|                47269|                      3|
| 105350500|        101260|       101259|                47269|                      3|
| 569763901|        101260|     

In [6]:
experiment_data_with_madcdl.count()

21398

In [7]:
experiment_data_with_madcdl.createOrReplaceTempView("experiment_data_with_madcdl")
experiment_data_with_madcdl_enriched = spark.sql("""
select b.*, a.module_id as original_module_id, a.instance_id as original_instance_id
from hp_grouped a inner join experiment_data_with_madcdl b 
on a.placement_id = b.original_PLMT
    
""")
experiment_data_with_madcdl_enriched.cache()
experiment_data_with_madcdl_enriched.show()

+----------+--------------+-------------+---------------------+-----------------------+------------------+--------------------+
|   user_id|replacement_id|original_PLMT|replacement_module_id|replacement_instance_id|original_module_id|original_instance_id|
+----------+--------------+-------------+---------------------+-----------------------+------------------+--------------------+
| 634081993|        101278|       101425|                47528|                      1|              3021|                   1|
| 802242081|        101278|       101425|                47528|                      1|              3021|                   1|
|  39423441|        101278|       101425|                47528|                      1|              3021|                   1|
|1099311390|        101260|       101259|                47269|                      3|             47269|                   2|
|1986943045|        101260|       101259|                47269|                      3|             4726

In [16]:
sql = """
select count(distinct user_id, replacement_module_id, replacement_instance_id)
from experiment_data_with_madcdl_enriched"""
spark.sql(sql).show()

+-----------------------------------------------------------------------+
|count(DISTINCT user_id, replacement_module_id, replacement_instance_id)|
+-----------------------------------------------------------------------+
|                                                                   9558|
+-----------------------------------------------------------------------+



23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 133 on hdc42-mcc10-01-0910-2206-012-tess0028.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 143 on hdc42-mcc10-01-0110-5606-004-tess0173.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 167 on hdc42-mcc10-01-0210-8007-016-tess0131.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 120 on hdc42-mcc10-01-0210-7502-020-tess0131.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 155 on hdc42-mcc10-01-0210-7802-008-tess0131.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 148 on hdc42-mcc10-01-0510-2002-033-tess0097.stratus.rno.ebay.com: Executor decommission.
23/06/25 13:20:33 ERROR YarnScheduler: Lost executor 113 on hdc42-mcc10-01-1110-2105-031-tess0029.stratus.rno.ebay.com: Executor decommission.

In [13]:
experiment_data_with_madcdl_enriched.createOrReplaceTempView("experiment_data_with_madcdl_enriched")
sql = """
select count(distinct a.user_id, replacement_module_id, replacement_instance_id)
from experiment_data_with_madcdl_enriched a join (
    select user_id, placement.placement_id as placement_id, mid as module_id, iid as instance_id
        FROM (
            select signedin_uid as user_id, mstat.*
            from ACCESS_VIEWS.HP_EVENT_FACT
            lateral view explode(module_stats5(module_info, module_lst, navsrc_lst, viewdtls_lst, module_plmt_imprns)) as mstat
            where dt = "20230525" AND type = "same_day"
        ) ms
        lateral view outer explode(placements) as placement
        where placement.placement_id is not null and iid <> -1
    ) b on a.replacement_module_id = b.module_id and a.replacement_instance_id = b.instance_id and a.user_id = b.user_id
"""
module_instance_fit = spark.sql(sql)
module_instance_fit.show()

+-----------------------------------------------------------------------+
|count(DISTINCT user_id, replacement_module_id, replacement_instance_id)|
+-----------------------------------------------------------------------+
|                                                                   9172|
+-----------------------------------------------------------------------+



In [22]:
percent = 9172/9558
print("Percent of (module_id, instance_id) that matches the replaced placement:", format(percent*100, ".2f"), "%")

Percent of (module_id, instance_id) that matches the replaced placement: 95.96 %


____

# Summary
```
By the MFE 1% EP investigating we can say that the replacement does happen.
The replaced placement can be found in the json MBE_VALUE in MADCDL.
The module_id and instance_id we see in HP_EVENT_FACT reflects the replaced placement (in the json).
```
